In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 파일 경로를 각자 컴퓨터 환경에 맞게 수정하세요!
file_path = '/content/drive/MyDrive/부리부리데마왕/dataset/'

In [ ]:
import numpy as np   # 결과해석
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]
import plotly.graph_objs as go
import plotly
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, StandardScaler

In [ ]:
pca_data = pd.read_csv(file_path + 'pca_train_test.csv')

In [ ]:
pca_data.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
0,-0.186803,-1.000143,0.745271,0.396115,-1.285372,-0.708153,0.114003,0.325531,0.592047,0.439203,...,0.884376,-0.557766,0.036501,0.809532,0.478148,0.135564,1.472248,0.428958,-0.381381,0
1,-0.705777,-0.671973,0.878802,0.384820,0.332147,0.265318,0.581809,-0.587116,0.875597,-0.961119,...,-0.536303,-0.159865,0.910140,0.123068,-0.891751,0.650127,0.867556,-0.841684,0.475563,0
2,3.973764,0.752199,-0.057570,0.722408,1.857562,-1.213780,0.526862,-1.335082,0.488178,-0.693094,...,0.431250,0.468316,0.554997,0.719100,-0.347534,0.322468,-0.220685,0.436515,-0.433111,0
3,1.025226,0.970736,0.634796,-0.730365,-0.767119,0.839303,0.244287,0.100426,0.462459,0.848337,...,-0.503240,0.372951,0.879522,0.369692,-0.584512,1.226589,-0.281516,-0.572916,0.434625,0
4,-0.892340,-0.479857,0.500884,-1.393628,0.103438,0.917956,-1.092323,0.107218,0.561216,0.387365,...,-0.379418,0.749823,0.204488,0.400505,0.618971,0.204516,-0.777073,0.554618,-0.069441,0


In [ ]:
pca_data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
0,-0.186803,-1.000143,0.745271,0.396115,-1.285372,-0.708153,0.114003,0.325531,0.592047,0.439203,...,0.884376,-0.557766,0.036501,0.809532,0.478148,0.135564,1.472248,0.428958,-0.381381,0
1,-0.705777,-0.671973,0.878802,0.384820,0.332147,0.265318,0.581809,-0.587116,0.875597,-0.961119,...,-0.536303,-0.159865,0.910140,0.123068,-0.891751,0.650127,0.867556,-0.841684,0.475563,0
2,3.973764,0.752199,-0.057570,0.722408,1.857562,-1.213780,0.526862,-1.335082,0.488178,-0.693094,...,0.431250,0.468316,0.554997,0.719100,-0.347534,0.322468,-0.220685,0.436515,-0.433111,0
3,1.025226,0.970736,0.634796,-0.730365,-0.767119,0.839303,0.244287,0.100426,0.462459,0.848337,...,-0.503240,0.372951,0.879522,0.369692,-0.584512,1.226589,-0.281516,-0.572916,0.434625,0
4,-0.892340,-0.479857,0.500884,-1.393628,0.103438,0.917956,-1.092323,0.107218,0.561216,0.387365,...,-0.379418,0.749823,0.204488,0.400505,0.618971,0.204516,-0.777073,0.554618,-0.069441,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284801,0.403169,0.402539,0.680315,-1.007532,2.098599,-1.277573,-0.646375,0.263997,0.781231,-0.454061,...,-0.410567,0.437024,0.627868,0.884708,0.772984,0.648746,-1.144199,0.589628,-0.273099,0
284802,-0.645635,-0.645760,0.370055,0.821711,-2.313645,-0.673857,0.365090,-1.252529,-0.886413,0.571019,...,-0.818363,0.809678,-2.978813,-3.835946,0.356766,-0.600699,-0.224977,0.767544,-0.854795,0
284803,-0.422941,0.331869,0.884928,0.401079,0.942990,-0.526874,-3.605100,0.052291,0.472184,0.005477,...,-0.103788,-0.729232,1.035139,0.383185,0.082624,-0.556574,-0.533770,0.598696,0.625018,0
284804,3.444423,-1.391055,-0.505879,0.415283,0.514673,-1.376723,-1.068739,0.129165,0.814837,0.994285,...,-0.546426,-0.742508,0.887634,-0.692838,0.431044,-3.674984,0.213256,0.937968,-0.285184,0


ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN

# ADASYN 모델 초기화 및 샘플링 전략 설정
adasyn = ADASYN(sampling_strategy={1: 250000}, random_state=42)


# 특징과 클래스 분리
X = pca_data.drop('target', axis = 1)
y = pca_data['target']


# ADASYN을 사용하여 오버샘플링
X_resampled, y_resampled = adasyn.fit_resample(X, y)

# 새로운 데이터프레임 생성
balanced_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='target')], axis=1)

Fraud = balanced_data[balanced_data['target']==1]
Normal = pca_data[pca_data['target']==0]
Fraud.shape

(249822, 29)

In [ ]:
data_ADASYN = pd.concat([Fraud, Normal], ignore_index=True)
data_ADASYN.shape

(534136, 29)

case 4: ADASYN + scale 1

In [ ]:
data_ADASYN_case4=data_ADASYN.copy()
#꼬리가 오른쪽으로 긴 변수
pro_skew = ['V3', 'V5', 'V7','V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24','V27', 'V28']
neg_skew = ['V2', 'V11', 'V19', 'V20', 'V21']
normal_col = ['V1', 'V4','V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26' ]

# pro_skew 로그변환 적용
# 변수 내의 최소값이 0 이하인 경우, 모든 값을 조금 올려서 로그 변환이 가능하도록 조정
if (data_ADASYN_case4[pro_skew].min() <= 0).any():  # .any()를 추가하여 Series 전체에서 하나라도 조건을 만족하는지 확인
    for col in pro_skew:
        if data_ADASYN_case4[col].min() <= 0:
            data_ADASYN_case4[col] += abs(data_ADASYN_case4[col].min()) + 0.1

for col in pro_skew:
    data_ADASYN_case4[col + '_log_transformed'] = np.log(data_ADASYN_case4[col])


# neg_skew는 음의 왜도.
# 제곱 변환 적용
for col in neg_skew:
 data_ADASYN_case4[col + '_squared_transformed'] = data_ADASYN_case4[col] ** 2

#normal_col
# 정규화할 변수 리스트
scaler = MinMaxScaler()
for col in normal_col:
    data_ADASYN_case4[col + '_minmax_transformed'] = scaler.fit_transform(data_ADASYN_case4[col].values.reshape(-1, 1)) # 2차원 배열로 바꿔서 실행

# 적용 다한 column은 drop

data_ADASYN_case4.drop(columns = pro_skew+neg_skew+normal_col, inplace = True)

print(data_ADASYN_case4.head())

X_case4 = data_ADASYN_case4.drop('target',axis=1)
Y_case4 = data_ADASYN_case4['target']

# 데이터 나누기
X_case4_train, X_case4_test, Y_case4_train, Y_case4_test = train_test_split(X_case4, Y_case4, test_size=0.3, random_state=25)

   target  V3_log_transformed  V5_log_transformed  V7_log_transformed  \
0       1            3.505565            3.383792            3.190363   
1       1            3.513058            3.367627            3.174770   
2       1            3.495578            3.322953            3.078943   
3       1            3.493652            3.336109            3.231477   
4       1            3.447307            3.366029            3.235569   

   V9_log_transformed  V10_log_transformed  V12_log_transformed  \
0            3.010109             3.238308             2.820128   
1            2.941991             3.237927             2.813773   
2            2.950504             3.221166             2.816597   
3            3.021727             3.225068             2.893456   
4            2.984441             3.220460             3.039289   

   V14_log_transformed  V15_log_transformed  V16_log_transformed  ...  \
0             2.758485             3.549007             2.903524  ...   
1           

Case 5 ADASYN + Scale 2

In [ ]:
def quantileTransform(data, columns):
    qt = QuantileTransformer()
    scaled_data = pd.DataFrame(qt.fit_transform(data[columns]), columns=columns)
    return scaled_data

def boxcox_transform(data, columns, lambda_value):
    if lambda_value == 0:
        return np.log(data[columns])
    else:
        return (data[columns] ** lambda_value - 1) / lambda_value

data_ADASYN_case5 = data_ADASYN.copy()

X_case5 = data_ADASYN_case5.drop('Class',axis=1)
Y_case5 = data_ADASYN_case5['Class']

left_skewed_cols = ['V3', 'V5', 'V7', 'V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24', 'V27', 'V28']
right_skewed_cols = ['V2', 'V11', 'V19', 'V20', 'V21']

for col in left_skewed_cols:
    # 음의 왜도를 가진 열의 최소값이 0보다 작거나 같은 경우 양수로 변환
    if X_case5[col].min() <= 0:
        X_case5[col] += abs(X_case5[col].min()) + 1

# train_scaler
smtrain_left_skewed = boxcox_transform(X_case5, left_skewed_cols, 0.01)
smtrain_right_skewed = quantileTransform(X_case5, right_skewed_cols)

train_transformed_sm = pd.concat([X_case5, smtrain_left_skewed.add_suffix('_box_Transform'), smtrain_right_skewed.add_suffix('_qt_Transform')], axis=1)

# 그 외 수치
# minmax? standard?
from sklearn.preprocessing import StandardScaler
s_columns = ['V1', 'V4', 'V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26']
standardScaler = StandardScaler()

for col in s_columns:
    train_transformed_sm[col + '_standard_Transform'] = standardScaler.fit_transform(train_transformed_sm[[col]])

#train_transformed --> 전처리가 진행된 데이터
train_transformed_sm.drop(columns =left_skewed_cols + right_skewed_cols + s_columns, inplace = True)

data_ADASYN_case5=train_transformed_sm
print(data_ADASYN_case5.head())


X_case5_train, X_case5_test, Y_case5_train, Y_case5_test = train_test_split(data_ADASYN_case5, Y_case5, test_size=0.3, random_state=25)

   V3_log_Transform  V5_log_Transform  V7_log_Transform  V9_log_Transform  \
0          3.535153          3.417148          3.230696          3.058220   
1          3.542428          3.401518          3.215723          2.993407   
2          3.525458          3.358365          3.123924          3.001495   
3          3.523590          3.371066          3.270216          3.069295   
4          3.478642          3.399973          3.274153          3.033772   

   V10_log_Transform  V12_log_Transform  V14_log_Transform  V15_log_Transform  \
0           3.276788           2.878018           2.819945           3.577354   
1           3.276422           2.872021           2.805949           3.581718   
2           3.260299           2.874686           2.749367           3.581587   
3           3.264051           2.947361           2.863836           3.581631   
4           3.259620           3.086047           2.826264           3.591951   

   V16_log_Transform  V17_log_Transform  ...  V21_

Case 6 scale 3

In [ ]:
data_ADASYN_case6=data_ADASYN.copy()

X_case6 = data_ADASYN_case6.drop('target',axis=1).values
Y_case6 = data_ADASYN_case6['target'].values

X_case6_train, X_case6_test, Y_case6_train, Y_case6_test = train_test_split(X_case6, Y_case6, test_size = 0.30, random_state = 25)
# X_TS_train['Time'] = scale.fit_transform(X_TS_train.reshape(1,-1))
# X_TS_test['Time'] = scale.fit_transform(X_TS_test.reshape(1,-1))

print(X_case6_train.shape,Y_case6_train.shape)
print(X_case6_test.shape,Y_case6_test.shape)

(373895, 28) (373895,)
(160241, 28) (160241,)


In [ ]:
from sklearn.naive_bayes import GaussianNB  # case 4


model = GaussianNB()   # 가우시안 naive Bayes
model.fit(X_case4_train , Y_case4_train)  # 가우시안을 SMOTE로 했을 때는 잘나옴.
# 내 생각엔 변수간의 독립성을 주어진 상태이므로 더 잘나오는 거라고 생각함.

GaussianNB()

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case4_test
predicted = model.predict(X_case4_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.71      0.34      0.46     85368
           1       0.53      0.84      0.65     74873

    accuracy                           0.57    160241
   macro avg       0.62      0.59      0.55    160241
weighted avg       0.62      0.57      0.55    160241

Accuracy: 0.5731554346265937
F1 Score: 0.6488917179142326


**pca_data를 적용하는 것이 더 결과값이 낮게 나옴. 이 분류에선 pca_data를 X_data로 적용하는 것이 좋다고는 말할 수 없을 것 같음.**

In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.6881743845475431
Specision 0.17060440363753326
Precision 0.5269942384498011
False Positive Rate: 0.6645230062786993
False Negative Rate :  0.3118256154524569
True Positive Rate: 0.844149426361973


In [ ]:
from sklearn.naive_bayes import BernoulliNB  # 베르누이 NB case 4


model = BernoulliNB(alpha=2.0)
model.fit(X_case4_train, Y_case4_train)

BernoulliNB(alpha=2.0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case4_test
predicted = model.predict(X_case4_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.53      1.00      0.70     85368
           1       0.00      0.00      0.00     74873

    accuracy                           0.53    160241
   macro avg       0.27      0.50      0.35    160241
weighted avg       0.28      0.53      0.37    160241

Accuracy: 0.5327038648036395
F1 Score: 0.0


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.0
Specision 0.9999065170940171
Precision 0.0
False Positive Rate: 8.199793833755037e-05
False Negative Rate :  1.0
True Positive Rate: 0.0


베르누이 나이브 베이즈 분류는 이 데이터에서는 적합하지 않은 모델이고, 가우시안 나이브 베이즈 분류는 pca데이터로는 맞지 않는 걸 알 수 있다. pca 자체가 한쪽으로 가중치를 줘서 하는 거이기 때문에 가우시안 나이브 베이즈 분류에서 적합하지 않기에 성능평가가 낮게 나온 것을 볼 수 있다.

In [ ]:
from sklearn.naive_bayes import GaussianNB  # case 5


model = GaussianNB()   # 가우시안 naive Bayes
model.fit(X_case5_train , Y_case5_train)  # 가우시안을 SMOTE로 했을 때는 잘나옴.
# 내 생각엔 변수간의 독립성을 주어진 상태이므로 더 잘나오는 거라고 생각함.

GaussianNB()

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case5_test
predicted = model.predict(X_case5_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.69      0.47      0.56     85368
           1       0.56      0.76      0.64     74873

    accuracy                           0.61    160241
   macro avg       0.63      0.62      0.60    160241
weighted avg       0.63      0.61      0.60    160241

Accuracy: 0.6073414419530582
F1 Score: 0.6445237906916307


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.5861119388415655
Specision 0.28340750158931977
Precision 0.5585191277697814
False Positive Rate: 0.5281604348233531
False Negative Rate :  0.41388806115843446
True Positive Rate: 0.761836710162542


In [ ]:
from sklearn.naive_bayes import BernoulliNB  # 베르누이 NB case 5


model = BernoulliNB(alpha=2.0)
model.fit(X_case5_train, Y_case5_train)

BernoulliNB(alpha=2.0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case5_test
predicted = model.predict(X_case5_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.53      1.00      0.70     85368
           1       0.00      0.00      0.00     74873

    accuracy                           0.53    160241
   macro avg       0.27      0.50      0.35    160241
weighted avg       0.28      0.53      0.37    160241

Accuracy: 0.5327225866039278
F1 Score: 0.0


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.0
Specision 0.999946579056319
Precision 0.0
False Positive Rate: 4.6855964764314496e-05
False Negative Rate :  1.0
True Positive Rate: 0.0


In [ ]:
from sklearn.naive_bayes import GaussianNB  # case 6


model = GaussianNB()   # 가우시안 naive Bayes
model.fit(X_case6_train , Y_case6_train)  # 가우시안을 SMOTE로 했을 때는 잘나옴.
# 내 생각엔 변수간의 독립성을 주어진 상태이므로 더 잘나오는 거라고 생각함.

GaussianNB()

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case6_test
predicted = model.predict(X_case6_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.69      0.51      0.59     85368
           1       0.57      0.73      0.64     74873

    accuracy                           0.62    160241
   macro avg       0.63      0.62      0.61    160241
weighted avg       0.63      0.62      0.61    160241

Accuracy: 0.616814672898946
F1 Score: 0.6415360904187002


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.5559040459737553
Specision 0.32454968893521385
Precision 0.5698565635403811
False Positive Rate: 0.48582607065879485
False Negative Rate :  0.4440959540262447
True Positive Rate: 0.7338426402040789


In [ ]:
from sklearn.naive_bayes import BernoulliNB  # 베르누이 NB case 6


model = BernoulliNB(alpha=2.0)
model.fit(X_case6_train, Y_case6_train)

BernoulliNB(alpha=2.0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case6_test
predicted = model.predict(X_case6_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.58      0.71      0.64     85368
           1       0.56      0.42      0.48     74873

    accuracy                           0.57    160241
   macro avg       0.57      0.56      0.56    160241
weighted avg       0.57      0.57      0.56    160241

Accuracy: 0.5735049082319756
F1 Score: 0.4780902051227223


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.34061306434237587
Specision 0.6375435310643528
Precision 0.5582365844524102
False Positive Rate: 0.2901672757942086
False Negative Rate :  0.6593869356576242
True Positive Rate: 0.4180679283586874


차라리, 스케일링 안하는 방법이 훨씬 좋게 나옴.